In [2]:
import ConnectionConfig as cc
from delta import DeltaTable

cc.setupEnvironment()
spark = cc.startLocalCluster("factRides")
spark.getActiveSession()

In [ ]:
cc.set_connectionProfile("default")
ride_src_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable", "rides") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 5000) \
    .load()

ride_src_df.show()



+------+-----------------+-----------------+-------------------+-------------------+---------+--------------+-----------+---------+
|rideid|       startpoint|         endpoint|          starttime|            endtime|vehicleid|subscriptionid|startlockid|endlockid|
+------+-----------------+-----------------+-------------------+-------------------+---------+--------------+-----------+---------+
|     1|(51.2083,4.44595)|(51.1938,4.40228)|2015-09-22 00:00:00|2012-09-22 00:00:00|      844|         13296|       4849|     3188|
|     2|(51.2174,4.41597)|(51.2188,4.40935)|2015-09-22 00:00:00|2012-09-22 00:00:00|     4545|         45924|       NULL|     NULL|
|     3|(51.2088,4.40834)|(51.2077,4.39846)|2015-09-22 00:00:00|2012-09-22 00:00:00|     3419|         25722|       2046|     1951|
|     4|(51.2023,4.41208)|(51.2119,4.39894)|2015-09-22 00:00:00|2012-09-22 00:00:00|     1208|         31000|       1821|     2186|
|     5|(51.1888,4.45039)|(51.2221,4.40467)|2015-09-22 00:00:00|2012-09-22 0

In [4]:
dim_date = spark.read.format("delta").load("spark-warehouse/date_dim")
dim_date.show()

+--------+----------+----+----------+--------+---------+------+----------+-------+
| date_sk|      date|year|month_name|month_nr| day_name|day_nr|is_weekday|quarter|
+--------+----------+----+----------+--------+---------+------+----------+-------+
|20090101|2009-01-01|2009|   January|       1| Thursday|     5|         Y|      1|
|20090102|2009-01-02|2009|   January|       1|   Friday|     6|         Y|      1|
|20090103|2009-01-03|2009|   January|       1| Saturday|     7|         N|      1|
|20090104|2009-01-04|2009|   January|       1|   Sunday|     1|         N|      1|
|20090105|2009-01-05|2009|   January|       1|   Monday|     2|         Y|      1|
|20090106|2009-01-06|2009|   January|       1|  Tuesday|     3|         Y|      1|
|20090107|2009-01-07|2009|   January|       1|Wednesday|     4|         Y|      1|
|20090108|2009-01-08|2009|   January|       1| Thursday|     5|         Y|      1|
|20090109|2009-01-09|2009|   January|       1|   Friday|     6|         Y|      1|
|200

In [5]:
dim_lock = spark.read.format("delta").load("spark-warehouse/lock_dim")
dim_lock.show()

+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|station_id|lock_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|        12|    217|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    218|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    219|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    220|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    221|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    222|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    223|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    224|       120|Schijnpoo

In [6]:
dim_vehicle = spark.read.format("delta").load("spark-warehouse/vehicle_dim")
dim_vehicle.show()

+----------+-------------------+
|biketypeid|biketypedescription|
+----------+-------------------+
|         1|          Velo Bike|
|         2|        Velo E-Bike|
|         3|               Step|
|         4|            Scooter|
+----------+-------------------+



In [7]:
dim_weather = spark.read.format("delta").load("spark-warehouse/weather_dim")
dim_weather.show()

+----------+------------+
|weather_id|weather_type|
+----------+------------+
|         1|  Unpleasant|
|         2|    Pleasant|
|         4|     Unknown|
|         3|     Neutral|
+----------+------------+



In [8]:
dim_user = spark.read.format("delta").load("spark-warehouse/user_dim_current")
dim_user.show()

+------+-------------------+--------------------+--------------------+--------+-------+--------------------+------------+--------------------+-------------------+-------------------+--------------------+-------+
|userid|               name|               email|              street|  number|zipcode|                city|country_code|             user_sk|          scd_start|            scd_end|                 md5|current|
+------+-------------------+--------------------+--------------------+--------+-------+--------------------+------------+--------------------+-------------------+-------------------+--------------------+-------+
| 30001|   de Ruijter Maria|Maria.de.Ruijter@...|            Kallodam| 50 0802|   9120|Beveren-Waas/Haas...|          BE|e408b9fe-27d8-4ff...|1990-01-01 00:00:00|2100-12-12 00:00:00|ca6acc87cdd2dbf67...|   true|
| 30002|    Dijkstra Myrthe|Myrthe.Dijkstra@t...|    Lange Repeldreef|    539 |   2970|s Gravenwezel/Sch...|          BE|9ef0cf0d-b32e-406...|1990-01-01

In [9]:
dim_weather.createOrReplaceTempView("dimWeather")
dim_user.createOrReplaceTempView("dimUser")
dim_lock.createOrReplaceTempView("dimLock")
dim_vehicle.createOrReplaceTempView("dimVehicle")
dim_date.createOrReplaceTempView("dimDate")

In [22]:
ride_src_df.createOrReplaceTempView("rides_source")
ridesFactFromSource = spark.sql("select src.rideid as ride_id, dd.date_sk, dl.lock_id as start_lock_id, dl2.lock_id as end_lock_id, md5(concat(ride_id, dd.date_sk)) as md5 \
                                from rides_source as src \
                                left outer join dimDate as dd on src.starttime = cast(dd.date as DATE) \
                                left outer join dimLock as dl on src.startlockid = dl.lock_id \
                                left outer join dimLock as dl2 on src.endlockid = dl2.lock_id")
ridesFactFromSource.show(100)
ridesFactFromSource.createOrReplaceTempView("factRides_new")

+-------+--------+-------------+-----------+--------------------+
|ride_id| date_sk|start_lock_id|end_lock_id|                 md5|
+-------+--------+-------------+-----------+--------------------+
|      1|20150922|         4849|       3188|bc8cab957d7e02da2...|
|      2|20150922|         NULL|       NULL|f70bc90e65cc45559...|
|      3|20150922|         2046|       1951|fbdba2ebbd15003b5...|
|      4|20150922|         1821|       2186|a21759453704a0ee8...|
|      5|20150922|         6382|       2700|05df43301dc60a215...|
|      6|20150922|         NULL|       NULL|085ad6bfc0c792608...|
|      7|20150922|         1388|       3401|25197ab7dbfabb306...|
|      8|20150922|         2572|         13|8f4fa6bcd20ac709f...|
|      9|20150922|           50|       2067|e504563d627927f8b...|
|     10|20150922|         NULL|       NULL|2ceceb8878013facf...|
|     11|20150922|          985|       2148|70e459d527aa6c08a...|
|     12|20150922|         2039|       3038|fbd0d70e6d8f11e8e...|
|     13|2